In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime

# Fetch stock ticker data
ticker = 'AAPL'
stock = yf.Ticker(ticker)
expiration_dates = stock.options

# Initialize list to collect options data
option_data = []

# Loop through each expiration date to fetch options data
for expiration in expiration_dates:
    options = stock.option_chain(expiration)
    options.calls['expirationDate'] = expiration
    options.puts['expirationDate'] = expiration
    option_data.append(options.calls)
    option_data.append(options.puts)

# Combine all options data into a single DataFrame
option_chain = pd.concat(option_data, ignore_index=True)

# Fetch current price of the stock
current_price = stock.history(period='1d')['Close'].iloc[0]

# Define the Black-Scholes-Merton (BSM) model function
def bsm_option_price(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'.")
    
    return option_price

# Risk-free interest rate
risk_free_rate = 0.05

# Prepare columns for fair value and over/underpriced indication
option_chain['fair_value'] = np.nan
option_chain['over_under_price'] = np.nan
option_chain['lastTradeDate'] = pd.to_datetime(option_chain['lastTradeDate']).dt.tz_localize(None)
option_chain['expirationDate'] = pd.to_datetime(option_chain['expirationDate']).dt.tz_localize(None)

# Calculate the fair value and over/underpriced status
for index, row in option_chain.iterrows():
    S = current_price
    K = row['strike']
    expiration = row['expirationDate']
    last_trade = row['lastTradeDate']
    T = (expiration - last_trade).days / 365.0
    sigma = row['impliedVolatility']
    option_type = 'call' if row['contractSymbol'][10] == 'C' else 'put'
    
    if T > 0:
        fair_value = bsm_option_price(S, K, T, risk_free_rate, sigma, option_type)
        option_chain.at[index, 'fair_value'] = fair_value
        
        if row['lastPrice'] > fair_value:
            option_chain.at[index, 'over_under_price'] = 'Overpriced'
        elif row['lastPrice'] < fair_value:
            option_chain.at[index, 'over_under_price'] = 'Underpriced'
        else:
            option_chain.at[index, 'over_under_price'] = 'Fairly Priced'

# Display the entire option_chain DataFrame
print(option_chain)


/tmp/ipykernel_400/3747127477.py:67: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Overpriced' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  option_chain.at[index, 'over_under_price'] = 'Overpriced'


           contractSymbol       lastTradeDate  strike  lastPrice     bid  \
0     AAPL240524C00100000 2024-05-22 17:04:21   100.0      91.24   89.30   
1     AAPL240524C00105000 2024-05-14 19:54:01   105.0      82.30   84.25   
2     AAPL240524C00115000 2024-04-29 17:14:13   115.0      59.90   74.35   
3     AAPL240524C00120000 2024-05-23 14:34:19   120.0      69.14   69.30   
4     AAPL240524C00125000 2024-05-23 14:34:19   125.0      64.16   64.25   
5     AAPL240524C00130000 2024-05-20 16:52:32   130.0      61.65   59.25   
6     AAPL240524C00135000 2024-05-14 19:57:05   135.0      52.98   54.25   
7     AAPL240524C00140000 2024-05-21 19:35:34   140.0      52.85   49.20   
8     AAPL240524C00145000 2024-05-20 17:59:15   145.0      46.75   44.20   
9     AAPL240524C00148000 2024-05-15 16:00:09   148.0      42.12   41.25   
10    AAPL240524C00150000 2024-05-21 17:34:29   150.0      42.50   39.25   
11    AAPL240524C00155000 2024-05-22 13:39:30   155.0      37.56   34.15   
12    AAPL24